In [ ]:
import tensorflow as tf
import os
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.learn.python.learn.datasets import base
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import random_seed
import time
from PIL import Image
from random import shuffle

In [ ]:
TRAIN = '/data/face_dataset/train/'
TEST = '/data/face_dataset/test/'
def read_images(direc, test=False):

    pids = os.listdir(direc)
    images = [] 
    labels = []
    for pid in pids:
        if test: 
            labels.append(pid.split('-')[0]) #TODO PID is a filename here 
            images.append(direc + pid)
        else: 
            for image_file in os.listdir(direc + pid):
                #imagefiles.append(image_file)
                labels.append(int(pid))
                images.append(direc + pid + '/' + image_file)

    #print(images)
    return np.array(images), np.array(labels)


train_images, train_labels = read_images(TRAIN)
test_images, test_labels = read_images(TEST, test=True)

In [ ]:
label_order = train_labels.copy()
label_order = set(label_order)
label_order = sorted(list(label_order))

In [ ]:
def writeTfrecords(toname, filenames, labels, train=True, shuf=True):
    def _bytes_feature(value):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

    def _int64_feature(value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    
    def _float32_feature(value):
        return tf.train.Feature(float_list=tf.train.FloatList(value=value))

    tfrecords_filename = toname

    writer = tf.python_io.TFRecordWriter(tfrecords_filename)

    c = list(zip(filenames, labels))
    if shuf: 
        shuffle(c)
    
    for img_path, label in c:

        img = np.array(Image.open(img_path))
                
        y_arr = np.zeros([398])
        if train:
            ind = label_order.index(label)
            #print(ind)
            y_arr[ind] = 1
        
        
        #print(y_arr.reshape(-1))

        img_raw = img.tostring()

        example = tf.train.Example(features=tf.train.Features(feature={
            'image_raw': _bytes_feature(img_raw),
            'labels_raw': _float32_feature(y_arr.reshape(-1))}))

        writer.write(example.SerializeToString())

    writer.close()



In [ ]:
# writeTfrecords("traindataset2.tfrecords", train_images, train_labels, shuf=True)


In [ ]:
# writeTfrecords("testdataset2.tfrecords", test_images, test_labels, train=False, shuf=False)

In [ ]:
def parse_tfrecords(example_proto):
    
    features = tf.parse_single_example(
      example_proto,
      # Defaults are not specified since both keys are required.
      features={
        'image_raw': tf.FixedLenFeature([], tf.string),
        'labels_raw': tf.FixedLenFeature([398], tf.float32)
        })
    
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.reshape(image, [64 * 64, 3])
    #print(image.shape)
        
    return image, features["labels_raw"]

In [ ]:
x = tf.placeholder(tf.float32, [None, 64 * 64, 3])
W = tf.Variable(tf.zeros([64 * 64, 3, 398]))
b = tf.Variable(tf.zeros([398]))
y_ = tf.placeholder(tf.float32, [None, 398])

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape, value):
    initial = tf.constant(value, shape=shape)
    return tf.Variable(initial)

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
W_conv1 = weight_variable([5, 5, 3, 32])
b_conv1 = bias_variable([32], 0.0)

In [ ]:
x_image = tf.reshape(x, [-1,64,64,3])

In [ ]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64], 1.0)

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [ ]:
W_fc1 = weight_variable([16 * 16 * 64, 1024])
b_fc1 = bias_variable([1024], 1.0)

h_pool2_flat = tf.reshape(h_pool2, [-1, 16*16*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [ ]:
#TODO 
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) #don't start with this, its hard 

In [ ]:
W_fc2 = weight_variable([1024, 398])
b_fc2 = bias_variable([398], 1.0)

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
#y_conv=tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)

In [ ]:
BATCH_SIZE = 128

traindata = tf.data.TFRecordDataset(['traindataset2.tfrecords'])
traindata = traindata.map(parse_tfrecords)
traindata = traindata.batch(BATCH_SIZE)

testdata = tf.data.TFRecordDataset(['testdataset2.tfrecords'])
testdata = testdata.map(parse_tfrecords)
testdata = testdata.batch(1024)

In [ ]:
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.maximum(y_conv + 1e-10, 1.0)), reduction_indices=[1]))

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.clip_by_value(y_conv, 1e-10, 1.0)), reduction_indices=[1]))


train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
itera = traindata.make_initializable_iterator()
next_element = itera.get_next()

itertest = testdata.make_initializable_iterator()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
#     for i in range(100):
    for i in range(100):
        sess.run(itera.initializer)
        its = 0 
        start_time = time.time()
        
        print("starting epoch " + str(i))
#         while True:
#             try:
#                 t = sess.run(next_element)
#                 #print(t[0])
#                 #print(sess.run(tf.argmax(y_conv, 1), feed_dict={x: t[0], keep_prob:1.0}))
                
#                 if its % 100 == 0:
#                     train_accuracy = accuracy.eval(feed_dict={
#                         x:t[0], y_: t[1], keep_prob:1.0})
#                     print("step %d, training accuracy %f" % (its, train_accuracy))
                    
                    
#                 train_step.run(feed_dict={x: t[0],  y_: t[1], keep_prob:0.5})

#                 its += 1
#             except tf.errors.OutOfRangeError:
#                 break
                
        print("testing and saving")
        saver.save(sess, '/home/colegrigsby/proj3/model' + str(i))
        predictions = []
        sess.run(itertest.initializer)
        while True:
            try: 
                tt = sess.run(itertest.get_next())
                p = sess.run(tf.argmax(y_conv, 1), feed_dict={x: tt[0], keep_prob: 1.0})
                if len(predictions):
                    predictions = np.concatenate((predictions, p))
                else:
                    predictions = p
                
            except tf.errors.OutOfRangeError:
                break

        
        #print(predictions)
        preds = [label_order[p] for p in predictions] 
        print(set(preds))
        #write predictions at each epoch 
        np.save('workfile' + str(i), preds)
        elapsed_time = time.time() - start_time
        print("end epoch: " + str(i) + " minutes run: " + str(elapsed_time/60))
        
    

In [ ]:
#TODO at end, sort things by the test labels 

In [ ]:
#use this 
import pandas as pd

In [ ]:
new_preds = np.load("workfiletry28.npy")
sub = pd.DataFrame()
sub["id"] = test_labels
sub["person"] = new_preds
sub["id"] = pd.to_numeric(sub["id"])
sub.sort_values("id").to_csv("subm2-8.csv", index=False)

In [ ]:
with tf.Session() as sess:    
    saver = tf.train.import_meta_graph('model5.meta')
    saver.restore(sess,'/home/colegrigsby/proj3/model5.data-00000-of-00001')
    predictions = []
    sess.run(itertest.initializer)
    while True:
        try: 
            tt = sess.run(itertest.get_next())
            p = sess.run(tf.argmax(y_conv, 1), feed_dict={x: tt[0], keep_prob: 1.0})
            if len(predictions):
                predictions = np.concatenate((predictions, p))
            else:
                predictions = p

        except tf.errors.OutOfRangeError:
            break


    #print(predictions)
    preds = [label_order[p] for p in predictions] 
    print(set(preds))
    #write predictions at each epoch 
    np.save('testing', preds)